In [1]:
# this notebook will estalish that a log-log-linear regression model is best suited for prediction of diamond prices from the Darden Diamond Price case
# the notebook 'Darden_Diamond_Price_Model_Optimization' will build on the log-log linear regression model

In [2]:
# import libraries for data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# import data sets
# training data
diamond_data_train = pd.read_csv('diamond_train.csv')

# holdout test data
diamond_data_holdout = pd.read_csv('diamond_test.csv')

In [4]:
diamond_data_train.columns

Index(['ID', 'Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry',
       'Report', 'Price'],
      dtype='object')

In [5]:
# analyze training data

# look for the unique values under the qualitative values under each columns
# will be useful for when using dummy variables for qualitative values
for col in diamond_data_train.iloc[:,2:8]:
    print(diamond_data_train[col].value_counts())

Ideal              2482
Very Good          2428
Good                708
Signature-Ideal     253
Fair                129
Name: Cut, dtype: int64
G    1501
H    1079
F    1013
I     968
E     778
D     661
Name: Color, dtype: int64
SI1     2059
VS2     1575
VS1     1192
VVS2     666
VVS1     285
IF       219
FL         4
Name: Clarity, dtype: int64
EX    2425
VG    2409
ID     595
G      571
Name: Polish, dtype: int64
VG    2417
EX    2059
G      916
ID     608
Name: Symmetry, dtype: int64
GIA     5266
AGSL     734
Name: Report, dtype: int64


In [6]:
# analyze training data
# look for the unique values under the qualitative values under each columns
for col in diamond_data_holdout.iloc[:,2:]:
    print(diamond_data_holdout[col].value_counts())

Ideal              1301
Very Good          1276
Good                373
Signature-Ideal     122
Fair                 70
Name: Cut, dtype: int64
G    752
H    631
F    522
I    480
E    411
D    346
Name: Color, dtype: int64
SI1     1051
VS2      821
VS1      634
VVS2     378
VVS1     166
IF        92
Name: Clarity, dtype: int64
EX    1279
VG    1235
G      323
ID     305
Name: Polish, dtype: int64
VG    1250
EX    1087
G      488
ID     317
Name: Symmetry, dtype: int64
GIA     2757
AGSL     385
Name: Report, dtype: int64


In [7]:
# training data, under 'Clarity' columns, has value FL, while holdout testing data does not
# remove the 4 sample rows of data that have value FL under 'Clarity' column
# new dimensions should have only 5996 rows
mask = diamond_data_train['Clarity'] == 'FL'
diamond_data_train_mask = diamond_data_train.loc[~mask]

# confirm new dimensions, with the 4 rows removed
diamond_data_train_mask.shape

(5996, 9)

In [8]:
# confirm dimensions of holdout data
diamond_data_holdout.shape

(3142, 8)

In [9]:
# training data will have an extra column because it will have the price column
# holdout test data will not have a price column
# must still remembers to drop 'ID' column from training and holdout data sets

In [10]:
# create the 'X' and 'y' variables for model training
# 'X' will drop 'ID' and 'Price columns'
X = diamond_data_train_mask.drop(['ID', 'Price'], axis=1)

# check data set
X.head()

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report
0,1.10,Ideal,H,SI1,VG,EX,GIA
1,0.83,Ideal,H,VS1,ID,ID,AGSL
2,0.85,Ideal,H,SI1,EX,EX,GIA
3,0.91,Ideal,E,SI1,VG,VG,GIA
4,0.83,Ideal,G,SI1,EX,EX,GIA


In [11]:
# 'X' dimensions check
X.shape

(5996, 7)

In [12]:
# 'y' will be just the price
y = diamond_data_train_mask['Price']

# check data set
y.head()

0    5169
1    3470
2    3183
3    4370
4    3171
Name: Price, dtype: int64

In [13]:
# 'y' dimensions check, ensure same amount of rows as 'X', 5996
y.shape

(5996,)

In [14]:
# get dummy variables for X to account for qualitative data
X_dummy = pd.get_dummies(X)

# inspect new dataframe
X_dummy.head()

,Carat Weight,Cut_Fair,Cut_Good,Cut_Ideal,Cut_Signature-Ideal,Cut_Very Good,Color_D,Color_E,Color_F,Color_G,...,Polish_EX,Polish_G,Polish_ID,Polish_VG,Symmetry_EX,Symmetry_G,Symmetry_ID,Symmetry_VG,Report_AGSL,Report_GIA
0,1.10,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
1,0.83,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
2,0.85,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
3,0.91,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
4,0.83,0,0,1,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1


In [15]:
# import libraries for training sample split up and linear regressiokn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [16]:
# split up training data for training validation purposes
# validation size will be 25% of training data
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y,
                                                   test_size=0.25, random_state=1
                                                   )

# testing first: Linear Model
# create linear regression object
reg_lin = LinearRegression()

# fit the data
reg_lin.fit(X_train, y_train)

# predict the data using X_test
y_lin_pred = reg_lin.predict(X_test)

# check accuracy of predictions between X_test and y_test
print('R^2: ' + str(reg_lin.score(X_test, y_test)))
print('RMSE: ' + str(mean_squared_error(y_test, y_lin_pred, squared=False)))

R^2: 0.8720977639626821
RMSE: 3494.6186197540483


In [17]:
# log-linear model now
# convert 'y' data to log values
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

# create new regression object for log-linear
reg_log_lin = LinearRegression()

# fit data; still use X_train, but now use y_train_log
reg_log_lin.fit(X_train, y_train_log)

# predict the data using X_test
y_log_lin_pred = reg_log_lin.predict(X_test)

# check accuracy of predictions between X_test and y_test
print('R^2: '+str(reg_log_lin.score(X_test, y_test_log)))
print('RMSE: ' + str(mean_squared_error(y_test_log, y_log_lin_pred, squared=False)))

R^2: 0.959647467807276
RMSE: 0.14159489323580388


In [18]:
y_train_log

1226    9.931005
325     8.883917
1878    8.557759
86      8.339979
2929    9.550876
          ...   
906     8.035603
5196    8.410053
3982    8.903408
236     8.525360
5161    8.746398
Name: Price, Length: 4497, dtype: float64

In [19]:
# make X log data for log-log model
# make a copy of X_train
X_train_log = X_train.copy()

# add new column that is the log of Carat Weight and inspect
X_train_log['log_CW'] = np.log(X_train_log.loc[:, 'Carat Weight'])
#X_train_log.head()

# has 29 columns now, drop the original 'Carat Weight column'
X_train_log = X_train_log.drop('Carat Weight', axis=1)

# inspect, should be back down to 28 columns
#X_train_log.head()

# repeat above for X_test dataframe
# make X log data for log-log model
# make a copy of X_test
X_test_log = X_test.copy()

# add new column that is the log of Carat Weight and inspect
X_test_log['log_CW'] = np.log(X_test_log.loc[:, 'Carat Weight'])

# has 29 columns now, drop the original 'Carat Weight column'
X_test_log = X_test_log.drop('Carat Weight', axis=1)

# inspect, should be back down to 28 columns
#X_test_log.head()

# create new regression object for log-log
reg_log_log = LinearRegression()

# fit data; use X_train_log alongside y_train_log
reg_log_log.fit(X_train_log, y_train_log)

# predict the data using X_test
y_log_log_pred = reg_log_log.predict(X_test_log)

# check accuracy of predictions between X_test and y_test
print('R^2: '+str(reg_log_log.score(X_test_log, y_test_log)))
print('RMSE: ' + str(mean_squared_error(y_test_log, y_log_log_pred, squared=False)))

R^2: 0.9803352109810775
RMSE: 0.09884548568094273


In [20]:
# log-log model has highest score
# use for holdout data set
# must prepare holdout data
# drop ID column first

X_holdout = diamond_data_holdout.drop('ID', axis=1)

# inspect dimensions (should have 7 columns)
#X_holdout.shape

In [21]:
# inspect dataframe
#X_holdout.head()

In [22]:
# get dummy variables for X_holdout
X_holdout_dummy = pd.get_dummies(X_holdout)

# inspect, should have 28 columns
#X_holdout_dummy.head()

# must now log transform the 'Carat Weight' columns and then drop the original CW column
# make copy to preserve original data
X_holdout_dummy_log = X_holdout_dummy.copy()

# add new column that is log of Carat Weight
X_holdout_dummy_log['log_CW'] = np.log(X_holdout_dummy_log.loc[:, 'Carat Weight'])

# inspect, should have 29 columns
#X_holdout_dummy_log.head()

# drop the original 'Carat Weight column'
X_holdout_dummy_log = X_holdout_dummy_log.drop('Carat Weight', axis=1)

# inspect; should have 28 columns again with 'log_CW' at the end
#X_holdout_dummy_log.head()

In [23]:
# holdout data prepared to enter model for testing
# use model: reg_log_log
y_holdout_log_log_pred = reg_log_log.predict(X_holdout_dummy_log)

# inspect y_holdout_log_log_pred
#y_holdout_log_log_pred

# must exp transform to get the price
output = np.exp(y_holdout_log_log_pred)

# inspect
#output

# inspect dimensions 
#output.shape

A3 = pd.DataFrame(output)
A3.columns = ['Price']
# outputs to CSV
#A3.to_csv('A3_predictions', index=None, header=None)

In [24]:
# inspect output
output

array([17426.49573356, 43357.98197008,  3778.87874834, ...,
        9850.84482969,  7599.14496717,  3010.57899176])